In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
import numpy as np
from torchvision import datasets, models, transforms
import torch.utils.data as data
import matplotlib.pyplot as plt

import pickle as pkl
import copy
import os

from sklearn.metrics import confusion_matrix
import sys
import utils

In [2]:
all_class = os.listdir("../Images/periodic/all")
print(len(all_class))

remove_classes = []
test_classes   = all_class[:5]
all_class = [ i for i in all_class if i not in remove_classes ]

train_target_classes = [ i for i in all_class if i not in test_classes ]
test_target_classes =  [ i for i in all_class if i in test_classes ]

print(len(train_target_classes))
print(len(test_target_classes))

55
50
5


In [3]:
train_packs = utils.get_paths(train_target_classes)
test_packs  = utils.get_paths(test_target_classes)
train_transforms = utils.data_transformer_torch_train()
test_transforms  = utils.data_transformer_torch_test()

In [4]:
datasets_train = utils.Img_Dataset(file_list=train_packs[0],transform=train_transforms,labels=train_packs[1],class_labels=train_packs[2])
datasets_test  = utils.Img_Dataset(file_list=test_packs[0] ,transform=test_transforms,labels=test_packs[1],class_labels=test_packs[2])

dataloader_train = torch.utils.data.DataLoader(datasets_train, batch_size=8, shuffle=True,num_workers=8)
dataloader_test  = torch.utils.data.DataLoader(datasets_test , batch_size=8, shuffle=False,num_workers=8)

dataloaders  = {"train":dataloader_train,"val":dataloader_test }
dataset_sizes ={"train":len(datasets_train),"val":len(datasets_test)}

In [5]:
model = models.vgg16(pretrained=True)
model.classifier = nn.Sequential(
        nn.Linear(25088,100),
        nn.ReLU(),
        nn.Dropout(0.5),
        nn.Linear(100,1) ,nn.Sigmoid())

In [6]:
#criterion = nn.CrossEntropyLoss()
criterion = nn.BCELoss()
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)
scheduler = lr_scheduler.StepLR(optimizer, step_size=10, gamma=0.5)

In [7]:
model,loss_dict,acc_dict = utils.training_model(dataloaders,dataset_sizes,model,criterion,
                     optimizer,scheduler,num_epochs=5)

Epoch 1/5
----------
---train---


/home/yusuke/anaconda3/envs/pytorch/lib/python3.6/site-packages/torch/optim/lr_scheduler.py:122: UserWarning: Detected call of `lr_scheduler.step()` before `optimizer.step()`. In PyTorch 1.1.0 and later, you should call them in the opposite order: `optimizer.step()` before `lr_scheduler.step()`.  Failure to do this will result in PyTorch skipping the first value of the learning rate schedule. See more details at https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate
  "https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate", UserWarning)


Traceback (most recent call last):
Traceback (most recent call last):
  File "/home/yusuke/anaconda3/envs/pytorch/lib/python3.6/multiprocessing/queues.py", line 240, in _feed
    send_bytes(obj)
  File "/home/yusuke/anaconda3/envs/pytorch/lib/python3.6/multiprocessing/connection.py", line 200, in send_bytes
    self._send_bytes(m[offset:offset + size])
  File "/home/yusuke/anaconda3/envs/pytorch/lib/python3.6/multiprocessing/connection.py", line 404, in _send_bytes
    self._send(header + buf)
  File "/home/yusuke/anaconda3/envs/pytorch/lib/python3.6/multiprocessing/connection.py", line 368, in _send
    n = write(self._handle, buf)
BrokenPipeError: [Errno 32] Broken pipe
  File "/home/yusuke/anaconda3/envs/pytorch/lib/python3.6/multiprocessing/queues.py", line 240, in _feed
    send_bytes(obj)
  File "/home/yusuke/anaconda3/envs/pytorch/lib/python3.6/multiprocessing/connection.py", line 200, in send_bytes
    self._send_bytes(m[offset:offset + size])
  File "/home/yusuke/anaconda3/env

KeyboardInterrupt: 

In [ ]:
with open("models/log.pkl","wb") as f:
    pkl.dump([loss_dict,acc_dict],f)

torch.save(model.state_dict(),"models/final_model_wts.pt")

In [10]:
loss,acc,labels,preds,classes = utils.test_model(dataloaders,dataset_sizes,model,criterion)
correct_index = labels == (preds > 0.5)
print(sum( correct_index )/100)
print( np.array(classes)[correct_index] )

Loss: 0.6983 ,ACC:0.4600
0.46
['Fabric.0008_s.o' 'Fabric.0008_s.o' 'Fabric.0008_s.o' 'sawtooth-wiggle.o'
 'd30_2180.o' 'd30_2180.o' 'd30_2180.o' 'd30_2180.o' 'd30_2180.o'
 'd30_2180.o' 'd30_2180.o' 'd30_2180.o' 'd30_2180.o' 'X100_0628.o'
 'X100_0628.o' 'checkerboard.o' 'checkerboard.o' 'checkerboard.o'
 'checkerboard.o' 'checkerboard.o' 'checkerboard.o' 'Fabric.0008_s.o'
 'Fabric.0008_s.o' 'sawtooth-wiggle.o' 'sawtooth-wiggle.o'
 'sawtooth-wiggle.o' 'sawtooth-wiggle.o' 'sawtooth-wiggle.o'
 'sawtooth-wiggle.o' 'sawtooth-wiggle.o' 'sawtooth-wiggle.o'
 'sawtooth-wiggle.o' 'sawtooth-wiggle.o' 'd30_2180.o' 'X100_0628.o'
 'X100_0628.o' 'X100_0628.o' 'X100_0628.o' 'X100_0628.o' 'X100_0628.o'
 'X100_0628.o' 'X100_0628.o' 'X100_0628.o' 'X100_0628.o' 'checkerboard.o'
 'checkerboard.o']
